In [0]:
!pip install polars 
!pip install optuna

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/ab/8a/6f56af7e535c34c95decc8654786bfce4632ba32817dc2f8bad18571ef9a/polars-1.32.3-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached polars-1.32.3-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (15 kB)
Using cached polars-1.32.3-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/5c/5e/068798a8c7087863e7772e9363a880ab13fe55a5a7ede8ec42fab8a1acbb/optuna-4.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/c2/62/96b5217b742805236614f05904541000f55422a6060a90d7fd4ce26c172d/alembic-1.16.4-py3-none-any.whl.metadata
  Obtaining dependency information for

In [0]:
import pandas as pd
import numpy as np
import polars as pl
import optuna
from optuna.exceptions import TrialPruned
from optuna.samplers import CmaEsSampler


In [0]:
# --- Utility and Metric Functions ---

def hitrate_at_3(y_true, y_pred, groups):
    """Hit rate @ 3 metric function using Polars."""
    df = pl.DataFrame({
        'group': groups,
        'pred': y_pred,
        'true': y_true
    })
    
    # Filter for groups with more than 10 items, as per the original logic
    # This prevents small groups from skewing the metric
    return (
        df.filter(pl.col("group").count().over("group") > 10)
        .sort(["group", "pred"], descending=[False, True])
        .group_by("group", maintain_order=True)
        .head(3)
        .group_by("group")
        .agg(pl.col("true").max())
        .select(pl.col("true").mean())
        .item()
    )

def stable_softmax(x):
    """Numerically stable softmax function."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def validate_brackets(brackets):
    """Validate that bracket definitions are properly formatted and non-overlapping."""
    if not brackets:
        raise ValueError("Brackets list cannot be empty")
    
    sorted_brackets = sorted(brackets, key=lambda x: x[0])
    
    for i, (min_size, max_size) in enumerate(sorted_brackets):
        if min_size >= max_size and max_size != float('inf'):
            raise ValueError(f"Bracket {i}: min_size ({min_size}) must be less than max_size ({max_size})")
        
        if i > 0:
            prev_max = sorted_brackets[i-1][1]
            if prev_max != float('inf') and min_size != prev_max:
                raise ValueError(f"Gap or overlap detected between bracket {i-1} and bracket {i}")
    
    print(f"✅ Bracket validation passed: {len(brackets)} brackets are properly defined.")
    return True

def assign_group_to_bracket(group_size, brackets):
    """Assign a group size to the appropriate bracket."""
    min_bracket_size = min(bracket[0] for bracket in brackets)
    if group_size < min_bracket_size:
        return "excluded_too_small"
    
    for i, (min_size, max_size) in enumerate(brackets):
        if i == len(brackets) - 1:
            if min_size <= group_size <= max_size:
                return f"bracket_{i}_{min_size}_{max_size if max_size != float('inf') else 'inf'}"
        else:
            if min_size <= group_size < max_size:
                return f"bracket_{i}_{min_size}_{max_size-1}"
    
    last_bracket = brackets[-1]
    return f"bracket_{len(brackets)-1}_{last_bracket[0]}_{last_bracket[1] if last_bracket[1] != float('inf') else 'inf'}"

# --- Optuna-based Optimizer ---

def find_optimal_weights_optuna(bracket_df, model_cols, hitrate_at_3, n_trials=100):
    """
    Finds optimal ensemble weights for a given bracket using Optuna.
    """
    num_models = len(model_cols)

    if bracket_df.empty:
        return 0.0, np.ones(num_models) / num_models

    def objective(trial):
        # Suggest a raw weight for each model between 0 and 1
        weights = [
            trial.suggest_float(f'weight_{model_col}', 0.0, 1.0) 
            for model_col in model_cols
        ]


        # Normalize the weights to enforce the sum-to-one constraint
        # normalized_weights = np.array(weights) / np.sum(weights)

        # Calculate the blended predictions
        blended_preds = np.zeros(len(bracket_df))
        for i, w in enumerate(weights):
            blended_preds += w * bracket_df[model_cols[i]]
        # Calculate and return the score for Optuna to maximize
        try:
            score = hitrate_at_3(
                y_true=bracket_df['selected'],
                y_pred=blended_preds,
                groups=bracket_df['group_key']
            )
            return score
        except Exception:
            # If any error occurs during metric calculation, return a very low score
            return 0.0

    # Create and run the Optuna study
    # The 'sampler' is what intelligently decides which weights to try next
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    
    # The 'n_jobs=-1' argument tells Optuna to use all available CPU cores
    study.optimize(objective, n_trials=n_trials, n_jobs=-1, show_progress_bar=True)

    # Extract the best weights found during the study
    best_params = study.best_params
    best_weights = np.array([best_params[f'weight_{model}'] for model in model_cols])

    # Normalize the final best weights to ensure they perfectly sum to 1
    # final_weights = best_weights / np.sum(best_weights)
    final_weights = best_weights
    return study.best_value, final_weights


In [0]:
def optimize_and_apply_ensemble(oof_df, test_dfs, model_cols, brackets):
    """
    Complete ensemble optimization and application pipeline using Optuna.
    This function now also calculates and returns the overall OOF score.
    """
    validate_brackets(brackets)

    # --- 1. OPTIMIZATION ON OOF DATA ---
    print("\n--- 🚀 Starting Bracketed Weight Optimization with Optuna ---")
    
    group_sizes = oof_df.groupby('group_key').size().reset_index(name='group_size')
    group_sizes['bracket'] = group_sizes['group_size'].apply(lambda x: assign_group_to_bracket(x, brackets))
    oof_with_brackets = pd.merge(oof_df, group_sizes[['group_key', 'bracket']], on='group_key')
    
    bracket_results = {}
    
    # Optimize weights for each bracket
    for bracket_name in sorted(oof_with_brackets['bracket'].unique()):
        if bracket_name == "excluded_too_small":
            continue

        print(f"\nOptimizing for: {bracket_name}...")
        bracket_data = oof_with_brackets[oof_with_brackets['bracket'] == bracket_name].copy()
        
        num_unique_groups = bracket_data['group_key'].nunique()
        print(f"  Number of unique groups in this bracket: {num_unique_groups}")        

        best_score, best_weights = find_optimal_weights_optuna(
            bracket_data, model_cols, hitrate_at_3, n_trials=1500,
        )
        
        bracket_results[bracket_name] = {
            'score': best_score,
            'weights': dict(zip(model_cols, best_weights))
        }
        
        print(f"  🏆 Best Score: {best_score:.5f}")
        for model, weight in bracket_results[bracket_name]['weights'].items():
            print(f"     - {model}: {weight:.4f}")

    # --- 2. EVALUATION ON ENTIRE OOF DATA ---
    print("\n--- 📈 Evaluating Blended Model on Full OOF Data ---")
    oof_with_brackets['blended_oof_pred'] = 0.0
    for bracket_name, results in bracket_results.items():
        mask = oof_with_brackets['bracket'] == bracket_name
        weights = results['weights']
        
        weighted_sum = np.zeros(mask.sum())
        for model_col, weight in weights.items():
            weighted_sum += weight * oof_with_brackets.loc[mask, model_col]
        
        oof_with_brackets.loc[mask, 'blended_oof_pred'] = weighted_sum
    
    overall_oof_score = hitrate_at_3(
        y_true=oof_with_brackets['selected'],
        y_pred=oof_with_brackets['blended_oof_pred'],
        groups=oof_with_brackets['group_key']
    )
    print(f"  Overall OOF Hitrate@3 with optimized weights: {overall_oof_score:.5f}")


    # --- 3. APPLICATION ON TEST DATA ---
    print("\n--- 🧪 Applying Bracketed Weights to Test Data ---")
    
    # Robustly merge test dataframes
    test_merged = test_dfs[model_cols[0]][['Id', 'ranker_id']].copy()
    for model_col in model_cols:
        if 'prediction' not in test_dfs[model_col].columns:
            raise ValueError(f"Input file for {model_col} is missing 'prediction' column.")
        pred_df = test_dfs[model_col][['Id', 'prediction']].rename(columns={'prediction': f'pred_{model_col}'})
        test_merged = pd.merge(test_merged, pred_df, on='Id', how='left')

    # Apply softmax
    for model_col in model_cols:
        test_merged[f'pred_{model_col}'] = test_merged.groupby('ranker_id')[f'pred_{model_col}'].transform(stable_softmax)

    # Assign brackets
    test_group_sizes = test_merged.groupby('ranker_id').size().reset_index(name='group_size')
    test_group_sizes['bracket'] = test_group_sizes['group_size'].apply(lambda x: assign_group_to_bracket(x, brackets))
    test_merged = pd.merge(test_merged, test_group_sizes[['ranker_id', 'bracket']], on='ranker_id')

    # Apply optimized weights
    test_merged['blended_pred'] = 0.0
    for bracket_name, results in bracket_results.items():
        mask = test_merged['bracket'] == bracket_name
        weights = results['weights']
        
        weighted_sum = np.zeros(mask.sum())
        for model_col, weight in weights.items():
            weighted_sum += weight * test_merged.loc[mask, f'pred_{model_col}']
        
        test_merged.loc[mask, 'blended_pred'] = weighted_sum

    # Generate final rankings
    test_merged['selected'] = test_merged.groupby('ranker_id')['blended_pred'].rank(method='first', ascending=False).astype(int)
    
    return test_merged[['Id', 'ranker_id', 'selected']], bracket_results, overall_oof_score


In [0]:
oof_model1 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0034/ensemble_oof_predictions.parquet')

# Define your brackets (copy from your code)
# brackets = [
#     (10, 14),(14,18), (18, 22),(22, 27), (27,32) ,(32, 37),(37,45),(45, 51), (51, 63),
#     (63, 76), (76, 101),(101, 126), (126, 161), (161, 200), (200, 251), (251, 351),(351, 501),(501, 625),(625, 750),(750, 1001), (1001,1750) ,(1750, 2501), (2501, 3500),
#     (3500, float('inf'))
# ]    

brackets = [
    (10, 14),(14,18), (18, 22),(22, 27), (27,32) ,(32, 37),(37,42),(42, 48), (48, 56),
    (56, 66), (66, 78),(78, 98), (98, 120), (120, 155), (155, 205), (205, 280),(280,361),(361, 501),(501, 625),(625, 750),(750, 1001), (1001,1500) ,(1500, 2501), (2501, 3500),
    (3500, float('inf'))
]  
group_sizes = oof_model1.groupby('ranker_id').size().reset_index(name='group_size')
group_sizes['bracket'] = group_sizes['group_size'].apply(lambda x: assign_group_to_bracket(x, brackets))

# Print groups per bracket
print(group_sizes['bracket'].value_counts().sort_index())

bracket_0_10_13          5056
bracket_10_66_77         4328
bracket_11_78_97         5217
bracket_12_98_119        4491
bracket_13_120_154       4942
bracket_14_155_204       4537
bracket_15_205_279       5075
bracket_16_280_360       5104
bracket_17_361_500       3447
bracket_18_501_624       2954
bracket_19_625_749        913
bracket_1_14_17          4669
bracket_20_750_1000      1168
bracket_21_1001_1499     1820
bracket_22_1500_2500      769
bracket_23_2501_3499       53
bracket_24_3500_inf       499
bracket_2_18_21          5232
bracket_3_22_26          5067
bracket_4_27_31          4745
bracket_5_32_36          4366
bracket_6_37_41          3516
bracket_7_42_47          3899
bracket_8_48_55          4521
bracket_9_56_65          4340
excluded_too_small      14811
Name: bracket, dtype: int64


In [0]:
if __name__ == "__main__":
    # --- Make sure you have installed optuna: !pip install optuna ---
    
    # 1. Load OOF predictions
    print("Loading OOF predictions...")
    # Ensure file paths are correct for your environment
    oof_model1 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0034/ensemble_oof_predictions.parquet')
    oof_model2 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0037/ensemble_oof_predictions.parquet')
    oof_model3 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0036/ensemble_oof_predictions.parquet')
    oof_model4 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0039/ensemble_oof_predictions.parquet')
    oof_model5 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0040/ensemble_oof_predictions.parquet') 
    oof_model6 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0041/ensemble_oof_predictions.parquet')        
    oof_model7 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0042/ensemble_oof_predictions.parquet')      
    oof_model8 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0043/ensemble_oof_predictions.parquet')   
    oof_model9 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0045/ensemble_oof_predictions.parquet') 
    oof_model10 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0046/ensemble_oof_predictions.parquet') 
    oof_model11 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0047/ensemble_oof_predictions.parquet') 
    oof_model12 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0049/ensemble_oof_predictions.parquet') 
    oof_model13 = pd.read_parquet('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0050/ensemble_oof_predictions.parquet') 

    # 2. Prepare and merge OOF data
    model_cols = ['oof_model1', 'oof_model2', 'oof_model3', 'oof_model4',
                  'oof_model5','oof_model6','oof_model7','oof_model8','oof_model9',
                  'oof_model10','oof_model11','oof_model12','oof_model13']
    all_oof_dfs = [oof_model1, oof_model2, oof_model3, oof_model4,
                   oof_model5,oof_model6,oof_model7,oof_model8,
                   oof_model9,oof_model10,oof_model11,oof_model12,oof_model13]
    
    for df, name in zip(all_oof_dfs, model_cols):
        df.rename(columns={'oof_prediction': name}, inplace=True)
    
    # Robustly merge OOF dataframes on 'Id'
    oof_merged = all_oof_dfs[0]
    for i, df_to_add in enumerate(all_oof_dfs[1:]):
        model_name_to_add = model_cols[i+1]
        oof_merged = pd.merge(oof_merged, df_to_add[['Id', model_name_to_add]], on='Id')
    
    oof_merged['group_key'] = oof_merged['ranker_id']

    # Apply Softmax to OOF predictions
    for col in model_cols:
        oof_merged[col] = oof_merged.groupby('ranker_id')[col].transform(stable_softmax)
        
    # 3. Load Test predictions
    print("\nLoading test predictions...")
    # Ensure file paths are correct
    test_model1 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0034/ensemble_test_predictions_ranked.csv')
    test_model2 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0037/ensemble_test_predictions_ranked.csv')
    test_model3 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0036/ensemble_test_predictions_ranked.csv')
    test_model4 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0039/ensemble_test_predictions_ranked.csv')
    test_model5 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0040/ensemble_test_predictions_ranked.csv')
    test_model6 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0041/ensemble_test_predictions_ranked.csv')   
    test_model7 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0042/ensemble_test_predictions_ranked.csv')   
    test_model8 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0043/ensemble_test_predictions_ranked.csv')           
    test_model9 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0045/ensemble_test_predictions_ranked.csv')           
    test_model10 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0046/ensemble_test_predictions_ranked.csv')  
    test_model11 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0047/ensemble_test_predictions_ranked.csv')  
    test_model12 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0049/ensemble_test_predictions_ranked.csv')  
    test_model13 = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/runs/NB0050/ensemble_test_predictions_ranked.csv')  

    test_dfs = dict(zip(model_cols, [test_model1, test_model2, test_model3, test_model4,test_model5,test_model6,test_model7,test_model8,test_model9,test_model10,test_model11,test_model12,test_model13]))

    # 4. Define brackets
    # brackets = [
    #     (10, 14),(14,18), (18, 22),(22, 27), (27, 35),(35,45),(45, 51), (51, 63),(63, 76), (76, 101),
    #     (101, 126), (126, 161), (161, 200), (200, 251), (251, 351),(351, 501),
    #     (501, 625),(625, 750),(750, 1001), (1001,1750) ,(1750, 2501), (2501, 3500),
    #                                                                    (3500, float('inf'))
    # ]    

    # brackets = [
    #     (10, 14),(14,18), (18, 22),(22, 27), (27,32) ,(32, 37),(37,45),(45, 51), (51, 63),
    #     (63, 76), (76, 101),(101, 126), (126, 161), (161, 200), (200, 251), (251, 351),(351, 501),(501, 625),(625, 750),(750, 1001), (1001,1750) ,(1750, 2501), (2501, 3500),
    #     (3500, float('inf'))
    # ]    

    brackets = [
        (10, 14),(14,18), (18, 22),(22, 27), (27,32) ,(32, 37),(37,42),(42, 48), (48, 56),
        (56, 66), (66, 78),(78, 98), (98, 120), (120, 155), (155, 205), (205, 280),(280,361),(361, 501),(501, 625),(625, 750),(750, 1001), (1001,1500) ,(1500, 2501), (2501, 3500),
        (3500, float('inf'))
    ]  
    
    # 5. Run the full pipeline!
    final_output, results, overall_oof_score = optimize_and_apply_ensemble(
        oof_df=oof_merged,
        test_dfs=test_dfs,
        model_cols=model_cols,
        brackets=brackets
    )

    print("\n--- ✅ Process Complete ---")
    print(f"\nFinal Overall OOF Score: {overall_oof_score:.5f}")
    print("\nFinal Test Output Head:")
    print(final_output.head())

Loading OOF predictions...

Loading test predictions...
✅ Bracket validation passed: 25 brackets are properly defined.

--- 🚀 Starting Bracketed Weight Optimization with Optuna ---

Optimizing for: bracket_0_10_13...


[I 2025-08-16 20:05:06,883] A new study created in memory with name: no-name-7608f993-764f-4329-ab17-880e09e33255


  Number of unique groups in this bracket: 5056


  0%|          | 0/1500 [00:00<?, ?it/s]

[I 2025-08-16 20:05:07,083] Trial 0 finished with value: 0.8844992450931052 and parameters: {'weight_oof_model1': 0.7161312180505475, 'weight_oof_model2': 0.30150292411888613, 'weight_oof_model3': 0.36264443697688664, 'weight_oof_model4': 0.8602589583993161, 'weight_oof_model5': 0.6897017365597776, 'weight_oof_model6': 0.6796590638254937, 'weight_oof_model7': 0.3503082040281851, 'weight_oof_model8': 0.35122644283338944, 'weight_oof_model9': 0.44605006025357385, 'weight_oof_model10': 0.2591645225768584, 'weight_oof_model11': 0.354744475132006, 'weight_oof_model12': 0.6633039340376014, 'weight_oof_model13': 0.37026256993501194}. Best is trial 0 with value: 0.8844992450931052.
[I 2025-08-16 20:05:07,084] Trial 6 finished with value: 0.8860090588827378 and parameters: {'weight_oof_model1': 0.012640258511483538, 'weight_oof_model2': 0.8660576629967675, 'weight_oof_model3': 0.6147578200372658, 'weight_oof_model4': 0.16929938475757889, 'weight_oof_model5': 0.6685078919048769, 'weight_oof_mode

...

[I 2025-08-16 22:14:58,806] Trial 1 finished with value: 0.6854678168546782 and parameters: {'weight_oof_model1': 0.6400083933466687, 'weight_oof_model2': 0.05599471912315468, 'weight_oof_model3': 0.6002555925145677, 'weight_oof_model4': 0.26068193205698065, 'weight_oof_model5': 0.7882274715631341, 'weight_oof_model6': 0.6533207047994231, 'weight_oof_model7': 0.4477325836633387, 'weight_oof_model8': 0.3749687571386159, 'weight_oof_model9': 0.8270242140757367, 'weight_oof_model10': 0.017051981048012088, 'weight_oof_model11': 0.4789674744200898, 'weight_oof_model12': 0.7791742107749132, 'weight_oof_model13': 0.6092797326247542}. Best is trial 1 with value: 0.6854678168546782.
[I 2025-08-16 22:14:58,863] Trial 2 finished with value: 0.6848042468480424 and parameters: {'weight_oof_model1': 0.8391802839508947, 'weight_oof_model2': 0.1244756549073115, 'weight_oof_model3': 0.3161234387560766, 'weight_oof_model4': 0.3155854631974836, 'weight_oof_model5': 0.18983927261437017, 'weight_oof_model6

[I 2025-08-16 22:18:59,971] A new study created in memory with name: no-name-8b75da0d-816e-40dd-ab4f-b6bcfae83bd2


  Number of unique groups in this bracket: 4340


  0%|          | 0/1500 [00:00<?, ?it/s]

[I 2025-08-16 22:19:00,339] Trial 1 finished with value: 0.6981566820276498 and parameters: {'weight_oof_model1': 0.0014392355479979635, 'weight_oof_model2': 0.2320300226023163, 'weight_oof_model3': 0.9724045381110158, 'weight_oof_model4': 0.4812550747176958, 'weight_oof_model5': 0.7794658933732972, 'weight_oof_model6': 0.6009688486453149, 'weight_oof_model7': 0.637503671014615, 'weight_oof_model8': 0.7551337159468758, 'weight_oof_model9': 0.8506431839886277, 'weight_oof_model10': 0.45906139406571633, 'weight_oof_model11': 0.11062998413489689, 'weight_oof_model12': 0.6118748379136415, 'weight_oof_model13': 0.24043206344001256}. Best is trial 1 with value: 0.6981566820276498.
[I 2025-08-16 22:19:00,374] Trial 4 finished with value: 0.6965437788018434 and parameters: {'weight_oof_model1': 0.8142950964702832, 'weight_oof_model2': 0.25165468515460776, 'weight_oof_model3': 0.9789553283139112, 'weight_oof_model4': 0.59578715174675, 'weight_oof_model5': 0.9230985490949152, 'weight_oof_model6'

In [0]:
display(final_output)

Id,ranker_id,selected
18144679,c9373e5f772e43d593dd6ad2fa90f67a,50
18144680,c9373e5f772e43d593dd6ad2fa90f67a,13
18144681,c9373e5f772e43d593dd6ad2fa90f67a,209
18144682,c9373e5f772e43d593dd6ad2fa90f67a,103
18144683,c9373e5f772e43d593dd6ad2fa90f67a,42
18144684,c9373e5f772e43d593dd6ad2fa90f67a,211
18144685,c9373e5f772e43d593dd6ad2fa90f67a,364
18144686,c9373e5f772e43d593dd6ad2fa90f67a,66
18144687,c9373e5f772e43d593dd6ad2fa90f67a,45
18144688,c9373e5f772e43d593dd6ad2fa90f67a,239


In [0]:
import pandas as pd
final_output.to_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/ensemble_test_predictions_ranked_pd.csv',index=False)

In [0]:
import pandas as pd
# final_output.to_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/ensemble_test_predictions_ranked_pd.csv',index=False)

_ = pd.read_csv('/Workspace/Users/rajneesh.tiwari@tide.co/aero/ensemble_test_predictions_ranked_pd.csv')


In [0]:
display(_)